# **Anomaly Detection in Time Series Data using Ensemble Models**

This notebook demonstrates how to build and evaluate individual anomaly detection models and then create an ensemble of these models for improved performance.

## **Table of Contents**

1. [Data Preparation](#data-preparation)
2. [Build and Evaluate Individual Models](#individual-models)
   - [Matrix Profile](#matrix-profile)
   - [Isolation Forest](#isolation-forest)
   - [Autoencoder](#autoencoder)
   - [Convolutional Neural Network (CNN)](#cnn)
3. [Build and Evaluate Ensemble Model](#ensemble-model)
4. [Analyze Results](#analyze-results)
5. [Optimize Models and Ensemble](#optimize-models)
6. [Conclusion](#conclusion)


In [1]:
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from tensorflow.keras.models import load_model
import keras

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

from commons import create_sequences, load_data, add_advanced_time_features, add_lag_features, filter_and_save_data, scale_data, scale_selected_columns, preprocess_train_data, preprocess_eval_data, plot_anomalies
from commons import timesteps, data_file_path, time_column, target_column, train_data_split, train_data_path, eval_data_path, anomaly_trashold   # Variables which are shared across the application

ImportError: cannot import name 'preprocess_train_data' from 'commons' (C:\Users\b50578\OneDrive - FH CAMPUS 02\Documents\PhD\ProjectsSandbox\Ensemble\commons.py)

In [ ]:
def plot_reconstruction_error(data, time_column, reconstruction_error, threshold, title="Reconstruction Error with Anomalies"):
    """
    Plots the reconstruction error over time and highlights points exceeding the anomaly threshold.
    
    Parameters:
    - data: pd.DataFrame containing the dataset
    - time_column: str, name of the time column
    - reconstruction_error: np.array, reconstruction error values
    - threshold: float, anomaly threshold
    - title: str, plot title
    """
    
    fig, ax = plt.subplots(figsize=(14, 6))

    # Plot the reconstruction error over time
    ax.plot(data[time_column], reconstruction_error, label='Reconstruction Error', color='blue')

    # Highlight anomalies (points where reconstruction error exceeds the threshold)
    anomalies = data[reconstruction_error > threshold]
    ax.scatter(anomalies[time_column], reconstruction_error[reconstruction_error > threshold], 
               color='red', label='Anomalies', marker='x')

    # Add threshold line
    ax.axhline(y=threshold, color='r', linestyle='--', label='Threshold')

    # Add title and labels
    ax.set_title(title)
    ax.set_xlabel('Time')
    ax.set_ylabel('Reconstruction Error')
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_combined_anomalies_only(data, time_column, value_column, predictions, title="Anomalies Detected"):
    """
    Plots the data with anomalies highlighted.
    
    Parameters:
    - data: pd.DataFrame containing the dataset
    - time_column: str, name of the time column
    - value_column: str, name of the value column (e.g., power output)
    - predictions: np.array, combined predictions from both models (1 for normal, -1 for anomaly)
    - title: str, plot title
    """
    
    fig, ax = plt.subplots(figsize=(14, 6))

    # Plot the actual data values over time
    ax.plot(data[time_column], data[value_column], label='Data', color='blue')

    # Highlight anomalies in red
    anomalies = data[predictions == -1]
    ax.scatter(anomalies[time_column], anomalies[value_column], color='red', label='Anomalies', marker='x')

    # Add title and labels
    ax.set_title(title)
    ax.set_xlabel('Time')
    ax.set_ylabel(value_column)
    plt.legend()
    plt.grid(True)
    plt.show()


def plot_combined_anomalies(data, time_column, value_column, ensemble_type, title=None):
    """
    Plots the data and highlights anomalies detected by different models whose predictions are stored in the DataFrame columns.

    Parameters:
    - data: pd.DataFrame, the original data with time and value columns, and prediction columns
    - time_column: str, name of the time column
    - value_column: str, name of the column with values to plot
    - ensemble_type: str, the type of ensemble method used
    - title: str or None, title of the plot. If None, a default title is generated
    """
    import matplotlib.pyplot as plt

    # Ensure the time column is in datetime format
    data[time_column] = pd.to_datetime(data[time_column])

    # Set default title if not provided
    if title is None:
        title = f"Anomalies Detected by Ensemble with {ensemble_type}"

    fig, ax = plt.subplots(figsize=(14, 6))

    # Plot the original data
    ax.plot(data[time_column], data[value_column], label='Original Data', color='blue')

    # Define colors and markers
    colors = ['orange', 'red', 'green', 'purple', 'cyan', 'magenta']
    markers = ['x', 'o', '^', 's', 'd', '*']

    # Get prediction columns dynamically
    prediction_columns = [col for col in data.columns if col.endswith('_prediction')]

    for idx, pred_col in enumerate(prediction_columns):
        anomalies = data[data[pred_col] == -1]
        model_label = pred_col.replace('_prediction', '').replace('_', ' ').title() + ' Anomalies'
        ax.scatter(
            anomalies[time_column], anomalies[value_column],
            color=colors[idx % len(colors)],
            marker=markers[idx % len(markers)],
            label=model_label
        )

    # Labels and title
    ax.set_title(title)
    ax.set_xlabel('Time')
    ax.set_ylabel(value_column)
    plt.legend()
    plt.grid(True)
    plt.show()


def load_models(models_info):

    models = {}
    scalers = {}

    for model_name, info in models_info.items():
        model_path = info['path']
        model_type = info['type']
        scaler_path = info.get('scaler')

        # Load the model
        if model_type == 'sklearn':
            models[model_name] = joblib.load(model_path)
        elif model_type == 'keras':
            models[model_name] = load_model(model_path)
        else:
            raise ValueError(f"Unsupported model type: {model_type}")

        # Load the scaler if it exists
        if scaler_path:
            scalers[model_name] = joblib.load(scaler_path)
        else:
            scalers[model_name] = None
    return models, scalers


def scale_selected_columns(data, feature_columns_to_scale, scaler):
    """
    Scales only the specified columns in the dataset using the provided scaler.

    Parameters:
    - data (pd.DataFrame): The dataset with features to scale.
    - feature_columns_to_scale (list): List of column names to be scaled.
    - scaler (object): An instance of a scaler (e.g., StandardScaler or MinMaxScaler).

    Returns:
    - data (pd.DataFrame): The dataset with only the specified columns scaled.
    """
    # Copy the dataset to avoid modifying the original data
    data_scaled = data.copy()

    # Apply the scaler to the selected columns
    data_scaled[feature_columns_to_scale] = scaler.transform(data[feature_columns_to_scale])

    return data_scaled



def make_predictions(models, preprocessed_data, data, ocsvm_threshold=anomaly_trashold):
    """
    Makes predictions using each model and stores the predictions in the data DataFrame.

    Parameters:
    - models: dict, contains loaded models
    - preprocessed_data: dict, contains preprocessed data for each model
    - data: pd.DataFrame, the original data where predictions will be stored
    - ocsvm_threshold: float, percentile threshold for OneClassSVM anomaly detection (default: 95th percentile)

    Returns:
    - data: pd.DataFrame, the data with predictions added
    """

    for model_name, model in models.items():
        prediction_col = f"{model_name}_prediction"

        # Make predictions based on model type
        if isinstance(model, (IsolationForest, OneClassSVM)):
            # For OneClassSVM, calculate decision function (anomaly scores) and apply dynamic threshold
            if isinstance(model, OneClassSVM):
                # Calculate decision function (anomaly scores)
                anomaly_scores = -model.decision_function(preprocessed_data)
                # Set a dynamic threshold based on the provided percentile
                threshold = np.percentile(anomaly_scores, ocsvm_threshold)
                # Label anomalies: -1 for anomaly, 1 for normal
                predictions = np.where(anomaly_scores > threshold, -1, 1)
            else:
                # For IsolationForest, we directly use the model's predictions (-1 for anomaly, 1 for normal)
                predictions = model.predict(preprocessed_data)

        elif isinstance(model, keras.Model):
          
            
            # Check if the model is deep_autoencoder or an seq_autoencoder based on its name
            if "autoencoder_deep" in model_name.lower():
                reconstructed_data = model.predict(preprocessed_data)  # Use non-sequence data if that’s how it was trained.
                reconstruction_error = np.mean(np.abs(preprocessed_data - reconstructed_data), axis=1)
                threshold = np.percentile(reconstruction_error, anomaly_trashold)
                predictions = np.where(reconstruction_error > threshold, -1, 1)
               
            elif "autoencoder_seq" in model_name.lower():

                if preprocessed_data.shape[0] >= timesteps:
                    preprocessed_data_seq  = create_sequences(preprocessed_data, timesteps)
                else:
                    raise ValueError("Not enough samples to create the required sliding windows.")
                # For Autoencoder, compute reconstruction error
                reconstructed_data = model.predict(preprocessed_data_seq)

                                
                if reconstructed_data.shape != preprocessed_data_seq.shape:
                    # The autoencoder returns only one timestep per sequence.
                    print("Using sequence-to-one error calculation (comparing last time step)")
                    # Compare only the last time step in each sequence with the model output.
                    reconstruction_error = np.mean(np.abs(preprocessed_data_seq[:, -1, :] - reconstructed_data), axis=1)
                else:
                    print("Using full sequence error calculation")
                    reconstruction_error = np.mean(np.abs(preprocessed_data_seq - reconstructed_data), axis=(1, 2))
                
                #reconstruction_error = np.mean(np.abs(preprocessed_data_seq - reconstructed_data), axis=(1, 2))
                # Set a dynamic threshold (e.g., 95th percentile)
                threshold = np.percentile(reconstruction_error, anomaly_trashold)
                # Label anomalies: -1 for anomaly, 1 for normal
                predictions_seq = np.where(reconstruction_error > threshold, -1, 1)
                
                # Create a full-length predictions array and pad the first (timesteps - 1) rows.
                predictions = np.ones(preprocessed_data.shape[0], dtype=int)  # default to 1 (normal)
                predictions[timesteps - 1:] = predictions_seq
        else:
            raise ValueError(f"Unsupported model instance: {model}")

        # Store predictions in the DataFrame
        data[prediction_col] = predictions

    return data

<a id='ensemble-model'></a>
##  Build and Evaluate Ensemble Model**


## Load all models

In [ ]:
# 1. Define the models dictionary
models_info = {
    'iso_forest': {
        'path': f'USModels/isolation_forest_model_{timesteps}.pkl',
        'type': 'sklearn',
        'scaler': f'USModels/scaler_iso_forest_{timesteps}.pkl'
        #'scaler': None
    },
    'autoencoder_seq': {
        'path': f'USModels/best_seq_autoencoder_model_{timesteps}.keras',
        'type': 'keras',
        'scaler': f'USModels/scaler_ae_{timesteps}.pkl'
    },
#    'lstm': {
 #       'path': f'USModels/lstm_forecaster_model_{timesteps}.keras',
#        'type': 'keras',
#        'scaler': f'USModels/scaler_ae_{timesteps}.pkl'
#    },
#    'autoencoder_deep': {
#        'path': f'USModels/best_deep_autoencoder_model_{timesteps}.keras',
#        'type': 'keras',
#        'scaler': f'USModels/scaler_ae_{timesteps}.pkl'
#   },
    'oc_svm': {
        'path': f'USModels/ocsvm_model_{timesteps}.pkl',
        'type': 'sklearn',
        'scaler': f'USModels/scaler_ocsvm_{timesteps}.pkl'
    }
}

# 2. Load the models and scalers dynamically
models, scalers = load_models(models_info)
print(models)

In [ ]:
last_x_days_data = preprocess_eval_data(
    eval_data_path=eval_data_path, 
    time_column=time_column, 
    target_column=target_column, 
    timesteps=timesteps, 
    scaler=scalers['oc_svm'] # the scaler is the sam for all
)
# Drop the 'Time' column before fitting the model, as IsolationForest only accepts numeric features
last_x_days_data_numeric = last_x_days_data.drop(columns=['Time'])
print(last_x_days_data_numeric.head())


# 5. Make predictions with each model
last_x_days_data = make_predictions(models, last_x_days_data_numeric, last_x_days_data)

In [ ]:
# 6. Combine predictions
# For ensemble methods, you can dynamically retrieve the predictions
prediction_columns = [col for col in last_x_days_data.columns if col.endswith('_prediction')]

# Convert predictions to numerical values (-1 for anomaly, 1 for normal)
predictions_matrix = np.column_stack([last_x_days_data[col].values for col in prediction_columns])

# Extract model names from prediction columns by removing the '_prediction' suffix
model_names = [col.replace('_prediction', '') for col in prediction_columns]


### Either using the majority voting 1st example or weighted averaging 2ond example

In [ ]:
# Majority Voting
sum_predictions = np.sum(predictions_matrix, axis=1)
combined_predictions_majority = np.where(sum_predictions <= -1, -1, 1)  # At least half models predict anomaly


# Weighted Averaging (Define your own weights)
#weights = np.full(len(prediction_columns), 1 / len(prediction_columns))  # Equal weights
# Define your weights for each model

weights_dict = {
    'iso_forest': 0.4,
    'autoencoder_seq': 0.4,
    'lstm': 0.3,  # Added weight for lstm
    'autoencoder_deep': 0.4,
    'oc_svm': 0.2
}
weights = np.array([weights_dict[model_name] for model_name in model_names])


combined_scores = np.dot(predictions_matrix, weights)
combined_predictions_weighted = np.where(combined_scores < 0, -1, 1)

# Add combined predictions to the DataFrame
ensemble_type = 'Majority Voting'  # 'Weighted Averaging', 'Stacking', etc.
#ensemble_type = 'Weighted Averaging'  # or 'Weighted Averaging'
if ensemble_type == 'Majority Voting':
    last_x_days_data['combined_prediction'] = combined_predictions_majority
else:
    last_x_days_data['combined_prediction'] = combined_predictions_weighted


In [ ]:
num_anomalies = (combined_predictions_weighted == -1).sum()
num_anomalies

In [ ]:
# Assuming anomalies are marked as -1 in 'combined_prediction'
num_anomalies = (last_x_days_data['combined_prediction'] == -1).sum()

# Print the number of anomalies detected
print(f"Number of anomalies detected: {num_anomalies}")


In [ ]:
# Now, simply call the function without passing predictions
plot_combined_anomalies(
    data=last_x_days_data,
    time_column='Time',
    value_column='Inv 1 AC-Leistung (W)',
    ensemble_type=ensemble_type
)

In [ ]:
# Call the function to plot combined anomalies
plot_combined_anomalies_only(
    data=last_x_days_data,                  # The DataFrame containing the dataset
    time_column='Time',                     # The name of the time column
    value_column='Inv 1 AC-Leistung (W)',   # The value column (e.g., power output)
    predictions=last_x_days_data['combined_prediction'].values,  # Combined predictions (1 for normal, -1 for anomaly)
    title="Detected Anomalies from ensamble in Power Output"  # Plot title (optional)
)


In [ ]:
# List of prediction method column names

prediction_methods = ["iso_forest_prediction", 
                      "autoencoder_seq_prediction", 
                      "oc_svm_prediction", 
                      "combined_prediction_majority", 
                      "combined_prediction_union", 
                      "combined_prediction_weighted", 
                      "combined_prediction_stacking_logistic_reg", 
                      "combined_prediction_stacking_random_for",
                      "combined_prediction_stacking_xgboost"]

# Loop through each prediction method and plot the anomalies
for method in prediction_methods:
    # Check if the current prediction column exists in the DataFrame
    if method in last_x_days_data.columns:
        # Create a dynamic title by formatting the method name
        title = (
            f"Detected Anomalies from {method.replace('_prediction', '').replace('_', ' ').title()} in Power Output"
        )
        # Call the plotting function with the dynamic predictions column
        plot_combined_anomalies_only(
            data=last_x_days_data,                  # The DataFrame containing the dataset
            time_column='Time',                     # The name of the time column
            value_column='Inv 1 AC-Leistung (W)',   # The value column (e.g., power output)
            predictions=last_x_days_data[method].values,  # Dynamically use the prediction values
            title=title                             # Dynamic title for the plot
        )
    else:
        print(f"Column '{method}' not found in the DataFrame.")

## Archive code dependent on labels which I dont have for unsupervized learning

<a id='analyze-results'></a>
## **4. Analyze Results**


<a id='conclusion'></a>
## **6. Conclusion**

- **Optimized Individual Models**: Hyperparameter tuning was performed on Matrix Profile, Autoencoder, and CNN models.
- **Improved Performance**: Each model showed performance improvements after optimization.
- **Enhanced Ensemble Model**: The ensemble model was updated with optimized models, leading to better overall performance.
- **Weighted Ensemble**: Assigning weights based on individual model performance further improved the ensemble's effectiveness.
- **Future Work**: Further optimization can be done using more sophisticated techniques like Bayesian Optimization, and additional models can be incorporated into the ensembleparameters.


